# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de ventanas

In [3]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [35]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id")

## Función generadora de features

In [59]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]])
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, how="right", left_index=True, right_index=True)       

---

## Cantidad de apariciones en subastas dentro de la ventana

In [41]:
def cantidad_apariciones_en_subastas(dataframe):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(index=str, columns={"date": "cantidad_apariciones"})

In [43]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_en_subastas, Xs)

---

## Entrenamiento: [18/4 , 20/4] ; Target: [21/4 , 23/4]

---

## Entrenamiento: [19/4 , 21/4] ; Target: [22/4 , 24/4]

---

## Entrenamiento: [20/4 , 22/4] ; Target: [23/4 , 25/4]

---

## Entrenamiento: [21/4 , 23/4] ; Target: [24/4 , 26/4]

---

## Entrenamiento: [22/4 , 24/4] ; Target: [25/4 , 27/4]

---